# Observations with unanimous consensus.

Output: data/feateng_03_unanimous_idxs.npy

Indices in train.csv (after cleaning) where there is unanimous consensus and the number of votes is greater than 1.

In [5]:
import numpy as np
import pandas as pd

# base_dir = "../../kaggle_data/hms"
base_dir = "../../data/hms"
# base_dir = "/kaggle/input/hms-harmful-brain-activity-classification"

df_traincsv = pd.read_csv(f'{base_dir}/train.csv')

In [6]:
#
# Cleaning.
#

eegs_with_nans = np.load("../EDA/02_eegs_with_nan_ids.npy")

# Counting NaN in each eeg.
eegs_with_nans_qty = np.array([], dtype=int)
for eeg_id in eegs_with_nans:
    eeg = pd.read_parquet(f'{base_dir}/train_eegs/{eeg_id}.parquet')
    qty = len(eeg[pd.isna(eeg).any(axis=1)].index)
    eegs_with_nans_qty = np.append(eegs_with_nans_qty, qty)

df_qtys = pd.DataFrame({'eeg_id': eegs_with_nans, 'quantity': eegs_with_nans_qty})

# Discarding eeg_ids with more than 5 NaNs.
eeg_ids_gt5 = df_qtys.loc[df_qtys.quantity > 5].eeg_id
idxs_to_remove = df_traincsv.loc[df_traincsv.eeg_id.isin(eeg_ids_gt5.values)].index
idxs = df_traincsv.index.difference(idxs_to_remove)
eeg_ids_clean = df_traincsv.iloc[idxs].eeg_id.unique()

In [7]:
df = df_traincsv.iloc[idxs]
df

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0


In [8]:
df.loc[:,'dummy_seizure'] = 0
df.loc[df.seizure_vote > 0, 'dummy_seizure'] = 1
df.loc[:,'dummy_lpd'] = 0
df.loc[df.lpd_vote > 0, 'dummy_lpd'] = 1
df.loc[:,'dummy_gpd'] = 0
df.loc[df.gpd_vote > 0, 'dummy_gpd'] = 1
df.loc[:,'dummy_lrda'] = 0
df.loc[df.lrda_vote > 0, 'dummy_lrda'] = 1
df.loc[:,'dummy_grda'] = 0
df.loc[df.grda_vote > 0, 'dummy_grda'] = 1
df.loc[:,'dummy_other'] = 0
df.loc[df.other_vote > 0, 'dummy_other'] = 1
df['sum_votes'] = df.dummy_seizure + df.dummy_lpd + df.dummy_gpd + df.dummy_lrda + df.dummy_grda + df.dummy_other
df

/tmp/ipykernel_14201/1900860189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'dummy_seizure'] = 0
/tmp/ipykernel_14201/1900860189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'dummy_lpd'] = 0
/tmp/ipykernel_14201/1900860189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,...,lrda_vote,grda_vote,other_vote,dummy_seizure,dummy_lpd,dummy_gpd,dummy_lrda,dummy_grda,dummy_other,sum_votes
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,...,0,0,0,1,0,0,0,0,0,1
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,...,0,0,0,1,0,0,0,0,0,1
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,...,0,0,0,1,0,0,0,0,0,1
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,...,0,0,0,1,0,0,0,0,0,1
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,...,3,0,0,0,0,0,1,0,0,1
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,...,3,0,0,0,0,0,1,0,0,1
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,...,3,0,0,0,0,0,1,0,0,1
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,...,3,0,0,0,0,0,1,0,0,1


In [9]:
idxs = df.loc[df.sum_votes == 1].index
len(idxs)

50760

<div class="alert alert-block alert-info"> <b>NOTE:</b> 50760 observations unanimous consensus.</div>


In [10]:
df_u = df_traincsv.loc[idxs]
df_u['sum_votes'] = df_u.seizure_vote + df_u.lpd_vote + df_u.gpd_vote + df_u.lrda_vote + df_u.grda_vote + df_u.other_vote
df_u.loc[df_u.sum_votes > 1]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,sum_votes
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0,3
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0,3
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0,3
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0,3
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106795,351917269,6,12.0,2147388374,6,12.0,4195677307,10351,LRDA,0,0,0,3,0,0,3
106796,351917269,7,14.0,2147388374,7,14.0,290896675,10351,LRDA,0,0,0,3,0,0,3
106797,351917269,8,16.0,2147388374,8,16.0,461435451,10351,LRDA,0,0,0,3,0,0,3
106798,351917269,9,18.0,2147388374,9,18.0,3786213131,10351,LRDA,0,0,0,3,0,0,3


<div class="alert alert-block alert-info"> <b>NOTE:</b> 46449 observations with more than 1 vote and unanimous consensus.</div>


In [11]:
np.save("../data/feateng_03_unanimous_idxs.npy",df_u.loc[df_u.sum_votes > 1].index)